<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### The Basics

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Portfolio Theory

Topics of interest include:

* basics
* portfolio return
* portfolio risk
* possible risk-return combinations
* optimal portfolios
 * maximum Sharpe portfolio
 * minimum risk portfolio
 * minimum allocation portfolio
 * Expected Utility
 * Time-Additive Expected Utility

## More Realistic Model Economy

**_Two dates only (today, tomorrow), three traded assets and three future states with equal probability._**

## Basics

### Financial Assets

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
M = np.array((
    (17, 10, 5),
    (20, 17, 2),
    (19, 0, 15)
)).T
M

In [ ]:
np.linalg.matrix_rank(M)

In [ ]:
M0 = np.array((10, 10, 10))
M0

### Probability

In [ ]:
noa = M.shape[1]
noa

In [ ]:
P = np.ones(noa) / noa
P

### Expectation

In [ ]:
np.dot(M.T, P)

In [ ]:
np.dot(P, M)  # expected values of the assets' payoffs

### Expected Return

In [ ]:
M

In [ ]:
R = M / M0 - 1
R

In [ ]:
np.dot(P, R)

### Volatility

In [ ]:
# R.std?

In [ ]:
R.std(axis=0)

In [ ]:
R.var(axis=0)

In [ ]:
R.var(axis=0) ** 0.5

In [ ]:
np.sqrt(R.var(axis=0))

## Portfolio Return

In [ ]:
phi = np.ones(noa) / noa
phi

In [ ]:
# expected return of the portfolio with weights phi
np.dot(np.dot(R, phi), P)

In [ ]:
# expected return of the portfolio with weights phi
np.dot(P, np.dot(R, phi))

In [ ]:
def port_return(phi):
    return np.dot(P, np.dot(R, phi))

In [ ]:
port_return(phi)

## Portfolio Volatility

In [ ]:
R.var(axis=0)

In [ ]:
# np.cov?

In [ ]:
np.cov(R.T, ddof=0)

In [ ]:
np.diag(np.cov(R.T, ddof=0))

In [ ]:
np.allclose(np.diag(np.cov(R.T, ddof=0)), R.var(axis=0))

In [ ]:
np.dot(phi, np.dot(np.cov(R.T, ddof=0), phi))  # portfolio variance

In [ ]:
np.sqrt(np.dot(phi, np.dot(np.cov(R.T, ddof=0), phi)))  # portfolio volatility

In [ ]:
def port_risk(phi):
    return np.sqrt(np.dot(phi, np.dot(np.cov(R.T, ddof=0), phi)))

In [ ]:
port_risk(phi)

## Mean-Variance Portfolios

In [ ]:
p = np.random.random((5000, noa))
p = (p.T / p.sum(axis=1)).T
p[:10].sum(axis=1)

In [ ]:
rr = np.array([(port_risk(phi), port_return(phi)) for phi in p])
rr[:5]

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr[:, 0], rr[:, 1], 'ro')
plt.xlabel('portfolio risk')
plt.ylabel('portfolio return');

## Optimal Portfolios

In [ ]:
from scipy.optimize import minimize

### Maximum Sharpe Ratio

In [ ]:
def sharpe(phi):
    return port_return(phi) / port_risk(phi)

In [ ]:
phi

In [ ]:
sharpe(phi)

In [ ]:
bnds = noa * [(0, 1)]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: phi.sum() - 1}

In [ ]:
opt = minimize(lambda phi: -sharpe(phi), phi, bounds=bnds, constraints=cons)
opt

In [ ]:
-opt['fun']  # maximal Sharpe ratio

In [ ]:
opt['x']  # optimal portfolio allocation

In [ ]:
np.dot(M, opt['x'])  # future payoff of optimal portfolio

### Minimum Risk

In [ ]:
opt = minimize(port_risk, phi, bounds=bnds, constraints=cons)
opt

In [ ]:
opt['fun']  # minimum risk attainable

In [ ]:
opt['x']  # minimum risk portfolio

### Minimum Allocation 

In [ ]:
bnds = noa * [(0.1, 1)]  # minimum allocation
bnds

In [ ]:
# bnds = noa * [(0.1, 0.5)]  # adding maximum allocation
# bnds

In [ ]:
opt = minimize(lambda phi: -sharpe(phi), phi, bounds=bnds, constraints=cons)
opt

In [ ]:
opt = minimize(port_risk, phi, bounds=bnds, constraints=cons)
opt

### Expected Utility

In [ ]:
def u(x):
    return np.sqrt(x)

In [ ]:
def EU(phi):
    c1 = np.dot(M, phi)
    return np.dot(u(c1), P)

In [ ]:
EU(phi)

In [ ]:
w = 10

In [ ]:
bnds = noa * [(0.15, 1.0)]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: np.dot(M0, phi) - w}

In [ ]:
opt = minimize(lambda phi: -EU(phi), phi, bounds=bnds, constraints=cons)
opt

In [ ]:
-opt['fun']  # maximum expected utility

In [ ]:
opt['x']  # optimal portfolio

In [ ]:
np.dot(M, opt['x'])  # future payoff

### Time-Additive Expected Utility

In [ ]:
M0 = np.array((1, 10, 10, 10))
M0

In [ ]:
kappa = 0.9

In [ ]:
def EU(phi):
    c0 = phi[0]
    c1 = np.dot(M, phi[1:])
    return u(c0) + kappa * np.dot(u(c1), P)

In [ ]:
bnds = [(0, None)] + 3 * [(0.0, None)]
bnds

In [ ]:
opt = minimize(lambda phi: -EU(phi), np.ones(4), bounds=bnds, constraints=cons)
opt

## `pandas` Approach

In [ ]:
import pandas as pd

In [ ]:
M_ = pd.DataFrame(M)
M_

In [ ]:
M_.columns = [f'asset_{a}' for a in range(3)]

In [ ]:
M_.index = [f'state_{s}' for s in range(3)]

In [ ]:
M_

**EXERCISE: Reimplement the whole analysis with the `pandas DataFrame` as the base data structure.**

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>